In [159]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb
import os
import os.path as path
import pandas as pd
#from clodius import 

import negspy.coordinates as nc

print(os.getcwd())

/Users/hawleyj/Documents/Davos/Results/2019-10-24_higlass


In [29]:
# load annotations
chrom_labels = {
    "top": Track(
        track_type="horizontal-chromosome-labels",
        server="http://higlass.io/api/v1",
        tileset_uuid="NyITQvZsS_mOFNlz5C2LJg",
        position="top"
    ),
    "left": Track(
        track_type="vertical-chromosome-labels",
        server="http://higlass.io/api/v1",
        tileset_uuid="NyITQvZsS_mOFNlz5C2LJg",
        position="left"
    )
}

gene_annots = {
    "top": Track(
        track_type="horizontal-gene-annotations",
        server="http://higlass.io/api/v1",
        tileset_uuid="P0PLbQMwTYGy-5uPIQid7A",
        position="top"
    ),
    "left": Track(
        track_type="vertical-gene-annotations",
        server="http://higlass.io/api/v1",
        tileset_uuid="P0PLbQMwTYGy-5uPIQid7A",
        position="left"
    )
}

ctcf_motifs = {
    "top": Track(
        track_type="bedlike",
        server="http://higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="top"
    ),
    "left": Track(
        track_type="bedlike",
        server="http://higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="left"
    )
}

hg38 = nc.get_chrominfo("hg38")

In [160]:
samples = ["PCa" + str(i) for i in [13266,13848,14121,19121,3023,33173,40507,51852,53687,56413,57054,57294,58215]]
cooler_files = [path.join("..", "..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts", s + ".mcool") for s in samples]
lowc_tilesets = {s: cooler(f) for s, f in zip(samples, cooler_files)}

resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]

lowc_heatmaps = {
    s: {
        "top": Track(
            track_type='horizontal-heatmap',
            position='top',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight"
            }
        ),
        "bottom": Track(
            track_type='horizontal-heatmap',
            position='bottom',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=150,
            options={
                "maxZoom": str(resolutions.index(10000)),
                "colorbarPosition": "topRight"
            }
        ),
        "left": Track(
            track_type='vertical-heatmap',
            position='left',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(10000))
            }
        ),
        "right": Track(
            track_type='vertical-heatmap',
            position='right',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=150,
            options={
                "maxZoom": str(resolutions.index(10000))
            }
        ),
        "centre": Track(
            track_type='heatmap',
            position='center',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=500,
            options={
                "maxZoom": str(resolutions.index(10000))
            }
        )
    } for s in samples
}

In [161]:
tad_files = [path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", s + ".40000bp.aggregated-domains.bedGraph") for s in samples]
#tad_testfile = path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "PCa3023.40000bp.aggregated-domains.bedGraph.beddb")
#tad_testfile = path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "test.bed.beddb")
#tad_testtile = beddb(tad_testfile)
# tad_clodius_files = [path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", s + ".40000bp.aggregated-domains.bedGraph") for s in samples[4:5]]
# tad_tilesets = {s: beddb(f) for s, f in zip(samples, tad_cfiles)}
#tad_rectangles = {s: Track(
#    track_type="horizontal-2d-rectangle-domains",
#    position="top",
#    tileset=tad_tilesets[s],
#    filetype="beddb"
#) for s in samples}
#tad_testtrack = Track(
#    track_type="horizontal-2d-rectangle-domains",
#    position="top",
#    tileset=tad_testtile,
#    filetype="bed2ddb"
#)
#tad_testtrack = Track(
#    track_type="bedlike",
#    position="top",
#    tileset=tad_testtile,
#    filetype="beddb"
#)

In [184]:
foxa1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 36000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 39000000, hg38)
    ],
)

display, server, viewconf = higlass.display([foxa1_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'GY54_3JNTCe0crRH1wFHUQ', 'tracks': {'top': [{'ty…

In [179]:
t2e_translocation = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["bottom"],
        lowc_heatmaps["PCa13848"]["centre"],
        lowc_heatmaps["PCa13848"]["left"],
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)
nont2e_translocation = View(
    tracks=[

        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["top"],
        lowc_heatmaps["PCa3023"]["centre"],
        lowc_heatmaps["PCa3023"]["left"],
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)

display, server, viewconf = higlass.display([t2e_translocation, nont2e_translocation])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'fkRiwNlsSGy449nV3nm0pQ', 'tracks': {'top': [{'ty…

In [192]:
ar_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chrX", 63500000, hg38),
        nc.chr_pos_to_genome_pos("chrX", 71500000, hg38)
    ],
)

display, server, viewconf = higlass.display([ar_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'R7AhBNVsRPawu87khHx5Dw', 'tracks': {'top': [{'ty…

In [190]:
myc_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr8", 126000000, hg38),
        nc.chr_pos_to_genome_pos("chr8", 130000000, hg38)
    ],
)

display, server, viewconf = higlass.display([myc_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'ERkIRNX7QkCod9vovd_pww', 'tracks': {'top': [{'ty…